In [ ]:
import torch
import math
import gpytorch

# %load_ext autoreload
# %autoreload 2

import preprocUtils
import preprocRandomVariables
import preprocLikelihoods
import preprocModels

import numpy
from torch import nn, optim
from torch.autograd import Variable
from gpytorch.kernels import RBFKernel, GridInterpolationKernel, Kernel
from gpytorch.means import ConstantMean
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.random_variables import GaussianRandomVariable

In [1]:
# Plotly 
import plotly
from plotly import graph_objs as plt_type

plot_in_browser = True
if plot_in_browser:
    from plotly.offline import iplot as plt
    plotly.offline.init_notebook_mode(connected=True)
else:
    def plt(*args, **kwargs):
        plotly.offline.plot(*args, **kwargs, auto_open=False)
    
# import nbimporter
# from preprocVisualisationTesting import *

In [ ]:
plt([plt_type.Scatter(x=[1, 2, 3], y=[3, 2, 6])])

In [ ]:
plotly.__version__


In [ ]:
a = 3

In [ ]:
plt([plt_type.Scatter(x=[1, 2, 3], y=[3, 2, 6])], filename='my-graph.html')
# We can also download an image of the plot by setting the image parameter
# to the image format we want


In [ ]:
help(plotly.offline.offline.image)

In [ ]:
help(plotly.offline.plot
)

In [2]:
plt([plt_type.Scatter(x=[1, 2, 3], y=[3, 2, 6])], image_width=3500, image_height=500, image='svg')

In [ ]:
plt([plt_type.Scatter(x=[1, 2, 3], y=[3, 2, 6])], output_type='file', image_filename='figures/bla',
     image='png')

In [3]:
import os

In [4]:
os.environ

environ({'MODULE_VERSION_STACK': '3.2.10', 'XDG_SESSION_ID': '11614', 'TERM': 'xterm-color', 'SHELL': '/bin/bash', 'SSH_CLIENT': '192.168.213.21 33856 22', 'CONDA_SHLVL': '1', 'CONDA_PROMPT_MODIFIER': '(preprocEnv) ', 'SSH_TTY': '/dev/pts/8', 'USER': 'gbohner', 'LD_LIBRARY_PATH': '/nfs/nhome/live/gbohner/torch/install/lib:/nfs/nhome/live/gbohner/torch/install/lib:', 'CONDA_EXE': '/nfs/nhome/live/gbohner/anaconda2/bin/conda', 'TMUX': '/tmp/tmux-5287/default,38161,0', 'PATH': '/nfs/nhome/live/gbohner/anaconda2/envs/preprocEnv/bin:/nfs/nhome/live/gbohner/anaconda2/envs/preprocEnv/bin:/nfs/nhome/live/gbohner/bin/julia:/nfs/nhome/live/gbohner/bin/julia-0.4.7/bin:/nfs/nhome/live/gbohner/anaconda2/bin:/nfs/nhome/live/gbohner/torch/install/bin:/nfs/nhome/live/gbohner/bin/julia:/nfs/nhome/live/gbohner/bin/julia-0.4.7/bin:/nfs/nhome/live/gbohner/anaconda2/bin:/nfs/nhome/live/gbohner/torch/install/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/b

In [ ]:
from subprocess import call
import json
import plotly

fig = {"data": [{"y": [1,2,1]}]}
call(['orca', 'graph', json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder), '--format''-o '])


In [ ]:
# Create a grid for testing
grid_bounds = [(0., 1.), (0., 1.)]
grid_size = 64
grid = torch.zeros(len(grid_bounds), grid_size)
for i in range(len(grid_bounds)):
    grid_diff = float(grid_bounds[i][1] - grid_bounds[i][0]) / (grid_size - 2)
    grid[i] = torch.linspace(grid_bounds[i][0] - grid_diff, grid_bounds[i][1] + grid_diff, grid_size)

In [ ]:
# Create inducing points at grid locations
inducing_points = torch.zeros(int(pow(grid_size, len(grid_bounds))), len(grid_bounds))
prev_points = None
for i in range(len(grid_bounds)):
    for j in range(grid_size):
        inducing_points[j * grid_size ** i : (j + 1) * grid_size ** i, i].fill_(grid[i, j])
        if prev_points is not None:
            inducing_points[j * grid_size ** i : (j + 1) * grid_size ** i, :i].copy_(prev_points)
    prev_points = inducing_points[: grid_size ** (i + 1), : (i + 1)]

In [ ]:
grid

In [ ]:
inducing_points

In [ ]:
#base_kernel_2d = RBFKernel(active_dims=0) + RBFKernel(active_dims=1)
#base_kernel_2d.kernels[0].log_lengthscale = torch.nn.Parameter(torch.tensor([-1.]))
#dict(base_kernel.named_parameters())

#imagesc(RBFKernel()(torch.tensor([0.1, 0.5]).unsqueeze(0), inducing_points).evaluate().reshape(grid_size,grid_size))

## Define novel center-symmetrized kernels

In [ ]:
import preprocKernels
from preprocKernels import SymmetriseKernelLinearly, SymmetriseKernelRadially

In [ ]:
# class SymmetriseKernelLinearly(Kernel):
#     def __init__(self, base_kernel_module, center, active_dims=None):
#         super(SymmetriseKernelLinearly, self).__init__(active_dims=active_dims)
#         self.base_kernel_module = base_kernel_module
        
#         self.register_parameter(name="center", parameter=torch.nn.Parameter(center.squeeze()))
        
#         # TODO: Check if base_kernel_module is defined on the same dimensions as center
        
#     def forward(self, x1, x2, **kwargs):
# #         print(self.center.shape)
# #         print(x1.shape)
#         if not self.center.shape[0] == x1.shape[-1]:
#             raise RuntimeError("The input dimension should be the same as the center point dimension")
            
#         return self.base_kernel_module(
#             (x1 - self.center.unsqueeze(0)).abs(), 
#             (x2 - self.center.unsqueeze(0)).abs(), 
#             **kwargs
#         )

In [ ]:
# class SymmetriseKernelRadially(Kernel):
#     def __init__(self, base_kernel_module, center, active_dims=None):
#         super(SymmetriseKernelRadially, self).__init__(active_dims=active_dims)
#         self.base_kernel_module = base_kernel_module
        
#         self.register_parameter(name="center", parameter=torch.nn.Parameter(center.squeeze()))
        
#         # TODO: Check if base_kernel_module is defined for 1-dimensional inputs (that our radial transform creates)
        
#     def forward(self, x1, x2, **kwargs):
#         """
#         Seemingly kernel inputs need to be 3 dimensional ( ? x num_points x num_dimensions),
#         if any of these dimensions are dropped it doesn't work. ? is usually 1, I imagine it is num_batch
#         """
# #         print(self.center.shape)
# #         print(x1.shape)
# #         print(x2.shape)
#         if not self.center.shape[0] == x1.shape[-1]:
#             raise RuntimeError("The input dimension should be the same as the center point dimension")
            
# #         print((x1-self.center.unsqueeze(0)).shape) 
# #         print((x1-self.center.unsqueeze(0)).pow(2).sum(-1).sqrt().shape)
            
#         return self.base_kernel_module(
#             (x1-self.center.view(1,1,-1)).pow(2).sum(-1).sqrt().unsqueeze(-1), 
#             (x2-self.center.view(1,1,-1)).pow(2).sum(-1).sqrt().unsqueeze(-1), 
#             **kwargs
#         )

In [ ]:
# # Debug input dimensionality
# class PrintRBFKernel(RBFKernel):
#     def forward(self, x1, x2):
#         print(x1.shape)
#         print(x2.shape)
#         return super(PrintRBFKernel, self).forward(x1,x2)
    
# base_kernel_print = PrintRBFKernel()
# print(base_kernel_print(inducing_points[:,0]).evaluate().shape)
# print(base_kernel_print(inducing_points).evaluate().shape)

## Compare the new kernels

In [ ]:
RBFKernel().log_lengthscale.shape

In [ ]:
center = torch.tensor([0.5, 0.5])
point_eval = torch.tensor([0.1,0.3]).unsqueeze(0)

In [ ]:
point_loc_tuple = tuple((point_eval*grid_size).numpy().tolist()[0])

In [ ]:
# Non symmetrized-kernel
imagesc(RBFKernel()(point_eval, inducing_points).evaluate().reshape(grid_size, grid_size), 
        title='Non symmetrized-kernel, point at (%d, %d)' % point_loc_tuple)

In [ ]:
# Radially symmetrized kernel
a1 = SymmetriseKernelRadially(RBFKernel(), center)(point_eval, inducing_points).evaluate()
imagesc(a1.reshape(grid_size, grid_size), 
       title='Radially symmetrized-kernel, point at (%d, %d)' % point_loc_tuple)

In [ ]:
import plotly.offline

In [ ]:
plotly.offline.plot([plt_type.Heatmap(z=a2[0,:].reshape(grid_size, grid_size))], image_filename = 'test1', image='svg', image_height=1024, image_width=768)

In [ ]:
# XY symmetric Kernel
a2 = SymmetriseKernelLinearly(RBFKernel(), center)(point_eval, inducing_points).evaluate()
imagesc(a2[0,:].reshape(grid_size, grid_size),
      title='Linearly symmetrized-kernel, point at (%d, %d)' % point_loc_tuple)

# Add priors and individual output-scales to kernels

In [ ]:
from preprocUtils import toTorchParam, updateModelParams
from gpytorch.priors import SmoothedBoxPrior

In [ ]:
k1 = RBFKernel()

In [ ]:
k1.state_dict()

In [ ]:
updateModelParams(k1,{'log_lengthscale': k1.log_lengthscale+2})

In [ ]:
k1.state_dict()

In [ ]:
k1.register_parameter('log_lengthscale',k1.log_lengthscale, prior=SmoothedBoxPrior(math.exp(-3),math.exp(0.5),log_transform=True))

In [ ]:
k1.state_dict()

In [ ]:
k2 = RBFKernel()

In [ ]:
base_kernel = k1 + k2

In [ ]:
base_kernel.state_dict()

In [ ]:
k2.register_parameter('log_lengthscale',
                      k2.log_lengthscale, 
                      prior=SmoothedBoxPrior(math.exp(-3),math.exp(0.5),log_transform=True))

In [ ]:
base_kernel.state_dict()

In [ ]:
base_kernel.state_dict()

In [ ]:
base_kernel_with_weights = k1 + 0.5 * base_kernel.kernels[1]

In [ ]:
class ScaleKernel(Kernel):
    """
    Adds an extra tunable output-scale parameter to the kernel
    """
    def __init__(self, base_kernel, weight=1.0, weight_prior=None):
        super(ScaleKernel, self).__init__()
        self.base_kernel = base_kernel
        self.register_parameter(
            'log_outputscale',
            toTorchParam(weight, ndims=1, to_log=True),
            prior = weight_prior if weight_prior is not None else SmoothedBoxPrior(math.exp(-3),math.exp(1),log_transform=True)
        )
            
    def forward(self, *inputs, **kwargs):
        # Right-multiple (due to lazy-variables) with output scale
        return self.base_kernel(*inputs, **kwargs).mul(self.log_outputscale.exp())

In [ ]:
base_kernel = k1 + k2

In [ ]:
base_kernel_with_weights = k1 + ScaleKernel(k2, 0.3)

In [ ]:
base_kernel_with_weights(inducing_points).evaluate()

In [ ]:
base_kernel(inducing_points).evaluate()

In [ ]:
# Register parameter of base_kernels
k3 = preprocKernels.SymmetriseKernelRadially(ScaleKernel(RBFKernel()), center=torch.tensor([0.5,0.5]))

In [ ]:
k3.state_dict()

In [ ]:
# Try direct naming
k3.register_parameter('base_kernel_module.base_kernel.log_lengthscale',
                     toTorchParam(0.53, ndims=1),
                     prior = SmoothedBoxPrior(-3, -1))

In [ ]:
# Above doesn't work, have to manually navigate to relevant kernel submodule to register prior
k3.base_kernel_module.base_kernel.register_parameter('log_lengthscale',
                     toTorchParam(0.53, ndims=1),
                     prior = SmoothedBoxPrior(-3, -1))

In [ ]:
k3.state_dict()

In [ ]:
k1

In [ ]:
base_kernel = ScaleKernel(k1)+ScaleKernel(k2)+ScaleKernel(k3)

In [ ]:
for name, param, prior in base_kernel.named_parameter_priors():
    print name, prior.log_prob(param)

In [ ]:
a = gpytorch.Module()

In [ ]:
a.covar_module = base_kernel

In [ ]:
a.register_parameter('log_outputscale', toTorchParam(3.0), prior=SmoothedBoxPrior(-4,1))

In [ ]:
for name, param, prior in a.named_parameter_priors():
    print name, prior.log_prob(param)

# Test Symmetrized Spectral Mixture Kernel

For locally guassian and long-term anti-correlated

In [ ]:
np.log((1./100)/np.sqrt(2*np.pi**2))

In [ ]:
a= gpytorch.kernels.SpectralMixtureKernel(n_mixtures=2, n_dims=1)

x1= torch.tensor(0.).view(1,1,1)
x2 = torch.arange(200.).view(1,-1,1)

a.log_mixture_weights.data[0] = -100.
a.log_mixture_weights.data[1] = 0.
a.log_mixture_means.data[0]= 0.
a.log_mixture_means.data[1]= 1./200 # Frequency of cosyne
a.log_mixture_scales.data[0] = 0.#np.log((1./10)/np.sqrt(2*np.pi**2))
a.log_mixture_scales.data[1] = np.log((a.log_mixture_means.data[1]*2.)/np.sqrt(2*np.pi**2)) # log Precision of RBF (from 0), compute as log((1./expected_std)/2.)
plot(a.forward(x1, x2).view(-1,1))

In [ ]:
class MexicanHatKernel(Kernel):
    def __init__(self, lengthscale=1.):
        super(MexicanHatKernel, self).__init__()
        self.register_parameter(
            'log_lengthscale',
            preprocUtils.toTorchParam(lengthscale, ndims=1, to_log=True),
            prior = gpytorch.priors.SmoothedBoxPrior(-2.,1.,sigma=0.1)
        )
    def forward(self, x1, x2):
        distance = (x1.unsqueeze(-2) - x2.unsqueeze(-3)).abs()  # distance = x^(i) - z^(i)

        exp_term = distance.div(self.log_lengthscale.exp()).pow_(2).mul(-1.)
        cos_term = distance.div(self.log_lengthscale.exp()).mul_(math.pi)
        res = exp_term.exp_() * cos_term.cos_()

        return res.view(x1.size(0), x1.size(1), x2.size(1))

In [ ]:
a = MexicanHatKernel(lengthscale=150.)

In [ ]:
a.forward(x1,x2).shape

In [ ]:
plot(a.forward(x1,x2).view(-1,1))